In [ ]:
import requests
from pprint import pprint
import pandas as pd
import pymysql
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import uuid
sns.set()
%matplotlib inline

In [ ]:
db = pymysql.connect(host='localhost', user='root', database='admin_main', port = 3307, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

# Create a cursor object
cursorObject = db.cursor()

bags_Query = "select * from bags"
orders_Query = "select * from orders"
bagsent_Query =  "select * from bags_entries"
bagsentdet_Query = "select * from bags_entry_details"
ordersent_Query = "select * from orders_entries"
ordersentdet_Query = "select * from orders_entry_details"

cursorObject.execute(bags_Query)
cursorObject.execute(orders_Query)
cursorObject.execute(bagsent_Query)
cursorObject.execute(bagsentdet_Query)
cursorObject.execute(ordersent_Query)
cursorObject.execute(ordersentdet_Query)

bags_df = pd.read_sql(bags_Query, db)
orders_df = pd.read_sql(orders_Query, db)
bagsent_df = pd.read_sql(bagsent_Query, db)
bagsentdet_df = pd.read_sql(bagsentdet_Query, db)
ordersent_df = pd.read_sql(ordersent_Query, db)
ordersentdet_df = pd.read_sql(ordersentdet_Query, db)

In [ ]:
bags_new = bags_df.drop(bags_df.columns.difference(['id', 'subscriber_id', 'created_at']), 1)
orders_new = orders_df.drop(orders_df.columns.difference(['id', 'subscriber_id', 'created_at']), 1)
bagsent_new = bagsent_df.drop(bagsent_df.columns.difference(['id', 'bag_id', 'item_id', 'quantity', 'created_at']), 1)
bagsentdet_new = bagsentdet_df.drop(bagsentdet_df.columns.difference(['bag_entry_id', 'type', 'pos_code']), 1)
                                                                                                                                         
ordersent_new = ordersent_df.drop(ordersent_df.columns.difference(['id', 'order_id', 'item_id', 'created_at']), 1)
ordersentdet_new = ordersentdet_df.drop(ordersentdet_df.columns.difference(['order_entry_id', 'type', 'pos_code', 'price', 'quantity']), 1)

ordersentdet_new.shape

In [ ]:
bags_new.rename(columns={'id': 'bag_id'}, inplace=True)
orders_new.rename(columns={'id': 'order_id'}, inplace=True)
bagsentdet_new.rename(columns={'bag_entry_id': 'id'}, inplace=True)
ordersentdet_new.rename(columns={'order_entry_id': 'id'}, inplace=True)

bags = pd.merge(bagsent_new, bags_new, on='bag_id', how='inner')
bags2 = pd.merge(bagsentdet_new, bags, on='id', how='inner')
orders = pd.merge(ordersent_new, orders_new, on='order_id', how='inner')
orders2 = pd.merge(ordersentdet_new, orders, on='id', how='inner')
orders2.shape

In [ ]:
with_variance = ordersentdet_df.drop(ordersentdet_df.columns.difference(['type', 'name', 'price', 'quantity', 'created_at']), 1)
with_variance = with_variance[with_variance.type!='variance']

In [ ]:
start_date = "2020-03-01 00:00:00"
end_date = "2020-03-31 23:59:59"
bags_new = bags_new[bags_new.created_at.between(start_date,end_date)]

In [ ]:
writer = pd.ExcelWriter("/Users/AF/Downloads/Mcdo data/Mc messages, bags & orders.xlsx", engine='xlsxwriter')

bags_new.to_excel(writer, sheet_name='Bag Count')
bagsent_new.to_excel(writer, sheet_name='Bags')
orders2.to_excel(writer, sheet_name='Orders')

writer.save()